# Explanation of the Fama-French 3-Factor Model

The Fama-French 3-Factor Model was developed by Eugene Fama and Kenneth French in 1992 as an extension of the Capital Asset Pricing Model (CAPM). The primary purpose of the research was to better explain the variations in stock returns that CAPM failed to account for. CAPM, which uses only the market risk factor, was found to be insufficient in explaining the cross-section of expected stock returns.

## Components:

The Fama-French 3-Factor Model includes three factors:

### 1. Market Risk (Beta):

This is the same as the market risk factor in CAPM.

$$
R_m - R_f
$$


### 2. Size Factor (SMB: Small Minus Big):

This factor accounts for the excess returns of small-cap stocks over large-cap stocks.

$$
SMB = R_{\text{small}} - R_{\text{big}}
$$

### 3. Value Factor (HML: High Minus Low):

This factor accounts for the excess returns of value stocks (high book-to-market ratio) over growth stocks (low book-to-market ratio).

$$
HML = R_{\text{high}} - R_{\text{low}}
$$


## Model Equation:

The expected return of a portfolio according to the Fama-French 3-Factor Model is given by:

$$
E(R_i) = R_f + \beta_{\text{mkt}}(R_m - R_f) + \beta_{\text{SMB}} \cdot SMB + \beta_{\text{HML}} \cdot HML
$$

## Method 1: using regression to calculate the coefficients

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [3]:
def market_return(List, Start, End):
    j = 0 
    T = 250 
    N = len(List) 
    PORTFOLIO = np.zeros((T, N)) 
    for i in List:
        stock_symbol = yf.Ticker(i)
        data = stock_symbol.history(start=Start, end=End)
        Price = (data['Open'] + data['Close']) / 2 
        
        Return = Price.div(Price.shift(1)).dropna()  
        Return = np.log(Return) 
        
        PORTFOLIO[:,j] = Return
        j+=1

    return PORTFOLIO


In [4]:
Stock_symbols = ['JILL','ELTK','ONVO','UAVS', 'AEY']
Portfolio_Return = market_return(Stock_symbols, '2019-01-01', '2019-12-31') # portfolio return
[T,N] = Portfolio_Return.shape


In [5]:
Portfolio_Return_df = pd.DataFrame(Portfolio_Return, columns=Stock_symbols)
Portfolio_Return_df

,JILL,ELTK,ONVO,UAVS,AEY
0,-0.017544,0.013699,0.030615,-0.055570,0.025046
1,0.012702,0.015748,0.010000,-0.019231,-0.003540
2,0.030595,0.047939,0.019705,0.000000,0.010582
3,0.019581,0.041673,0.028848,0.047402,-0.003515
4,0.019205,0.006104,0.028039,0.027399,0.024349
...,...,...,...,...,...
245,0.037919,0.014916,-0.087969,-0.046520,-0.023331
246,0.036534,0.001345,-0.059189,-0.011976,-0.004301
247,-0.004494,-0.012170,0.012121,0.000000,0.033902
248,-0.009050,-0.005457,-0.049393,0.069796,0.000000


# To calculate the coeficient of the factors :

In [14]:
FAMA_FRENCH_3 = pd.read_csv("F-F_Research_Data_Factors_daily.csv",
                            names=['Mkt-RF','SMB','HML','RF'])
FAMA_FRENCH_3.head()

,Mkt-RF,SMB,HML,RF
20190102,0.23,0.57,1.10,0.01
20190103,-2.45,0.40,1.26,0.01
20190104,3.55,0.43,-0.72,0.01
20190107,0.94,0.96,-0.78,0.01
20190108,1.01,0.54,-0.64,0.01


In [7]:
# Converting series data to array
MKT = (FAMA_FRENCH_3['Mkt-RF'] - FAMA_FRENCH_3['RF']).to_numpy()
RF  = FAMA_FRENCH_3['RF'].to_numpy()

SMB = FAMA_FRENCH_3['SMB']
HML = FAMA_FRENCH_3['HML']


F = np.column_stack((np.ones((T)),MKT, SMB, HML))
K = F.shape[1]

### Volatility:

| Beta Value Range | Interpretation |
|------------------|----------------|
| $\beta = 1$      | The asset has the same level of risk as the market. If the market increases by 1%, the asset is also expected to increase by 1%, and vice versa. |
| $\beta > 1$      | The asset is more volatile than the market. For example, if $\beta = 1.2$, a 1% increase in the market would result in a 1.2% increase in the asset, and vice versa. |
| $0 < \beta < 1$  | The asset is less volatile than the market. For example, if $\beta = 0.8$, a 1% increase in the market would result in a 0.8% increase in the asset, and vice versa. |
| $\beta < 0$      | The asset moves in the opposite direction to the market. A negative $\beta$ means that the asset generally decreases in value if the market increases, and vice versa. |
| $\beta < -1$     | The asset is not only negatively correlated with the market but is also more volatile. For example, if $\beta = -1.2$, a 1% increase in the market would result in a 1.2% decrease in the asset. |



In [8]:
def get_beta(Portfolio_Return, RF, MKT):
    
    #Initializing beta with zero for each stock  
    beta = np.zeros((K,N))
    for i in range(0,N):
        y = Portfolio_Return[:,i]
        x = F
        beta[:,i] = np.linalg.inv(x.conj().T @ x) @ (x.conj().T) @ y
    return beta

In [9]:
beta = get_beta(Portfolio_Return, RF, MKT)
beta

array([[-0.00559659,  0.00210405, -0.00333122, -0.00138873,  0.00236407],
       [ 0.00495095, -0.00239557,  0.00184938,  0.00232735,  0.00043225],
       [ 0.00706509,  0.0173545 ,  0.01583944, -0.00318065,  0.00643096],
       [ 0.00467667, -0.0124349 ,  0.00718411, -0.00481578,  0.00139544]])

## Method 2: using OLS regression model to calculate the coefficients

In [12]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [13]:
X = FAMA_FRENCH_3

X1 = sm.add_constant(X) # Add a constant to the independent value

for i in range(0,5):
    y = Portfolio_Return[:,i]

    # make regression model 
    model = sm.OLS(y, X1)

    # fit model and print results
    results = model.fit()
    
    #Coeficient values: 
    print(f"{i+1}) Stock Symbol: ",Stock_symbols[i])
    print("\n Const: ",results.params[0])
    print(" Mkt-RF: ",results.params[1]," SMB: ",results.params[2]," HML: ",results.params[3]," RF: ",results.params[4])
    print("\n Summary: ")
    print(results.summary(),"\n")


1) Stock Symbol:  JILL

 Const:  -0.018443523276613822
 Mkt-RF:  0.004829116099156277  SMB:  0.007110103617912678  HML:  0.004457313885631877  RF:  1.5084792871833397

 Summary: 
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.7252
Date:                Tue, 12 Sep 2023   Prob (F-statistic):              0.575
Time:                        18:11:15   Log-Likelihood:                 339.00
No. Observations:                 250   AIC:                            -668.0
Df Residuals:                     245   BIC:                            -650.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std er